# Developing a pair potential for a dimer

In this exercise we will develop a simple potential for Cl<sub>2</sub> dimer. I pregenerated the reference data using the quantum chemistry program [NWChem](http://www.nwchem-sw.org/index.php/Main_Page) and density functional theory, aiming for speed instead of accuracy. THose data are stored in the file `e_vs_l.txt`. The first column is the distance between the two atoms in ångström and the second is the energy in hartree ($1\;\mathrm{Ha}\simeq 27.2114\;\mathrm{eV}$). Remember that the origin of energies is arbitrary: only the change in energy between two points matters.

If you are interested, you can compare our results with the experimental measurements in this very thorough paper:

<a href="Boris_M__Smirnov_1996_Phys.-Usp._39_R01.pdf" target="_blank">Boris M. Smirnov and A. S. Yatsenko *Phys.-Usp.* 39 (1996) 211</a>

In [ ]:
%matplotlib widget


from IPython.display import Markdown

import string

import numpy as np
import scipy as sp
import scipy.misc
import scipy.interpolate
import scipy.optimize
import matplotlib
import matplotlib.pyplot as plt

import ase
import ase.units

alphabet = string.ascii_lowercase
colors = matplotlib.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
# Load and plot the data from NWChem's output. Note that we use ase.units.Hartree to convert the energies to eV.
data = np.loadtxt("e_vs_l.txt")
r = data[:, 0]
E = data[:, 1] * ase.units.Hartree
# Set E=0 to the value of the energy for the dissociated molecule
E -= E[-1]

plt.figure()
plt.plot(r, E)
plt.axhline(y=0, ls="--", color="#666666", lw=2.)
plt.xlabel(r"$r\;(\mathrm{\AA})$")
plt.ylabel(r"$E_{\mathrm{pot}}\;(\mathrm{eV})$")
plt.show()

***

# Part A: Fitting a Lennard-Jones (LJ) potential

The Lennard-Jones potential is one of the oldest and simplest pair potentials. It combines a simple parametrization of the short-range part with the London (dipole correlation) attractive force:

$$
E_{\mathrm{pot}} = 4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right]
$$

There are thus two adjustable parameters in the LJ expression, namely $\epsilon$ and $\sigma$. There are several features of the DFT curve shown above that we may try to reproduce, including:

- The point where it crosses the $E_{\mathrm{pot}}=0$ axis, $r_{\mathrm{cross}}$
- The position of the minimum, $r_{\mathrm{min}}$
- The depth of the minimum, $E_{\mathrm{min}}$
- The fundamental vibrational frequency of the dimer, determined by $\left.\frac{d^2 E_{\mathrm{pot}}}{d r^2}\right|_{r=r_{\mathrm{min}}}$

To do so, we start by noting that $E_{\mathrm{pot}}\left(\sigma\right) = 0$, so $r_{\mathrm{cross}} = \sigma$. As for the position and value of the minimum, 

$$
\left.\frac{dE_{\mathrm{pot}}}{dr}\right|_{r=r_{\mathrm{min}}} = 24\left(\frac{r_{\mathrm{min}}}{\sigma}\right)^5 \left[2\left(\frac{r_{\mathrm{min}}}{\sigma}\right)^6  - 1\right]= 0 \Rightarrow r_{\mathrm{min}} = \frac{\sigma}{2^{1/6}};\;E_{\mathrm{pot}}\left(r_{\mathrm{min}}\right) = -\epsilon
$$

Finally,

$$
\left.\frac{d^2 E_{\mathrm{pot}}}{d r^2}\right|_{r=r_{\mathrm{min}}} = 24\epsilon \left(\frac{r_{\mathrm{min}}}{\sigma}\right)^4 \left[22\left(\frac{r_{\mathrm{min}}}{\sigma}\right)^6 - 5\right] = 72 \cdot 2^{1/3} \cdot\epsilon
$$

Any two of these relations will determine the values of $\epsilon$ and $\sigma$, and clearly they will not be strictly compatible for any given real $E_{\mathrm{pot}}\left(r\right)$. Moreover, matching any pair of features of the curves does not guarantee a good global fit. We explore these questions below by comparing the results of several possible reasonable choices.

In [ ]:
# Interpolate the results to a denser grid, for convenience
newr = np.linspace(r.min(), r.max(), num=1000)
newE = sp.interpolate.interp1d(r, E, kind="cubic")(newr)

# Find the position and depth of the minimum, and the position where the curve crosses zero
r_min = newr[newE.argmin()]
E_min = newE.min()
r_cross = newr[np.where(np.sign(newE[:-1]) != np.sign(newE[1:]))[0][0] + 1]

In [ ]:
def lj(r, sigma, epsilon):
    """Straightworward implementation of the Lennard-Jones potential."""
    rm6 = (sigma / r) ** 6
    return 4. * epsilon * rm6 * (rm6 - 1.)

## LJ (a): A potential that reproduces the position and depth of the minimum

In [ ]:
sigma1 = r_min / 2 ** (1. / 6.)
epsilon1 = -E_min
display(Markdown(r"""
LJ(a):

- $\sigma = {:.2f}\;\mathrm{{Å}}$
- $\epsilon = {:.2f}\;\mathrm{{eV}}$
""".format(sigma1, epsilon1)))

## LJ (b): The LJ potential affording the closest fit to all points of the curve below the horizontal axis

We get this one from a nonlinear least-squares fit that minimizes

$$
\int\limits_{r_{\mathrm{cross}}}^{\infty} \left\vert E_{\mathrm{pot}}\left(r\right) - 4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right] \right\vert^2 dr
$$

with respect to the parameters $\sigma$ and $\epsilon$.

In [ ]:
indices = (E < 0.)
result = scipy.optimize.curve_fit(lj, r[indices], E[indices],
                                  p0=(sigma1, epsilon1),
                                  bounds=((1., 0.), (3, 4.)))
sigma2, epsilon2 = result[0]
display(Markdown(r"""
LJ(b):

- $\sigma = {:.2f}\;\mathrm{{Å}}$
- $\epsilon = {:.2f}\;\mathrm{{eV}}$
""".format(sigma2, epsilon2)))

## LJ (c): A potential that reproduces the crossing point with the horizontal axis and the depth of the minimum

In [ ]:
# Fit another LJ potential using the crossing point and the depth of the minimum
sigma3 = r_cross
epsilon3 = -E_min
display(Markdown(r"""
LJ(c):

- $\sigma = {:.2f}\;\mathrm{{Å}}$
- $\epsilon = {:.2f}\;\mathrm{{eV}}$
""".format(sigma3, epsilon3)))

## LJ (d): A potential that reproduces the position of the minimum and the fundamental vibration frequency of the dimer

In [ ]:
# Fit yet another LJ potential using the position and the first derivative of the minimum
sigma4 = sigma1
Ktilde = sp.misc.derivative(sp.interpolate.interp1d(r, E),
                            2 ** (1. / 6.) * sigma4, dx=0.1, n=2,
                            order=3)
epsilon4 = 2 ** (1. / 3.) * sigma4 ** 2. * Ktilde / 72.
display(Markdown(r"""
LJ(a):

- $\sigma = {:.2f}\;\mathrm{{Å}}$
- $\epsilon = {:.2f}\;\mathrm{{eV}}$
""".format(sigma4, epsilon4)))

## Comparison of all results

In [ ]:
# Plot all of the results alongside the real thing
sigmas = [sigma1, sigma2, sigma3, sigma4]
epsilons = [epsilon1, epsilon2, epsilon3, epsilon4]

plt.figure()
plt.scatter(r, E, color=colors[0], label="DFT")
plt.plot(newr, newE, color=colors[0])
for i, (sigma, epsilon) in enumerate(zip(sigmas, epsilons)):
    plt.plot(newr, lj(newr, sigma, epsilon), color=colors[i + 1],
             label="LJ ({})".format(alphabet[i]))
plt.axhline(y=0., lw=1., color="#666666", ls=":")
plt.xlabel(r"$r\;\left(\mathrm{\AA}\right)$")
plt.ylabel(r"$E_{\mathrm{pot}}\;\left(\mathrm{eV}\right)$")
plt.legend(loc="best")
plt.xlim(1., 4.)
plt.ylim(-4., 4.)
plt.tight_layout()

#  Part B: Fitting a Morse potential

The Morse potential has the following functional form:

$$
E_{\mathrm{pot}} \left(r\right) = D_e\left\lbrace\left[1 - e^{-\alpha\left(r -r_c\right)}\right]^2 - 1\right\rbrace
$$

With three parameters ($D_e$, $\alpha$ and $r_c$) it offers a little more flexibility than the LJ model, and it is generally considered as a useful tool for a semi-quantitative description of a range of binary molecules.

We can easily obtain analytical expressions for the same features that we used in fitting the LJ potential:

- $$E_{\mathrm{pot}}\left(r_{\mathrm{cross}}\right) = 0 \Rightarrow r_{\mathrm{cross}} = r_c - \frac{\log 2}{\alpha}$$
- $$
\left.\frac{dE_{\mathrm{pot}}}{dr}\right|_{r=r_{\mathrm{min}}} = 2 \alpha D_e e^{-\alpha\left(r_{\mathrm{min}} -r_c\right)} \left[1 - e^{-\alpha\left(r_{\mathrm{min}} -r_c\right)}\right] = 0 \Rightarrow r_{\mathrm{min}} = r_c;\; E_{\mathrm{pot}}\left(r_{\mathrm{min}}\right) = -D_e
$$
- $$
\left.\frac{d^2 E_{\mathrm{pot}}}{d r^2}\right|_{r=r_{\mathrm{min}}} = 2 \alpha^2 D_e e^{-\alpha\left(r_{\mathrm{min}} -r_c\right)} \left[2 e^{-\alpha\left(r_{\mathrm{min}} -r_c\right)} - 1\right] = 2\alpha^2 D_e
$$


In [ ]:
def morse(r, De, alpha, rc):
    """Straightworward implementation of the Morse potential."""
    exp = np.exp(-alpha * (r - rc))
    return De * ((1. - exp)**2. - 1.)

## Morse (a): A potential that reproduces the crossing point, and the position and depth of the minimum

In [ ]:
De1 = -E_min
alpha1 = np.log(2) / (r_min - r_cross)
rc1 = r_min
display(Markdown(r"""
Morse (a):

- $r_c = {:.2f}\;\mathrm{{Å}}$
- $\alpha = {:.2f}\;\mathrm{{Å^{{-1}}}}$
- $D_e = {:.2f}\;\mathrm{{eV}}$
""".format(rc1, alpha1, De1)))

## Morse (b): The Morse potential affording the closest fit to all points of the curve below the horizontal axis

We get this one from a nonlinear least-squares fit that minimizes

$$
\int\limits_{r_{\mathrm{cross}}}^{\infty} \left\vert E_{\mathrm{pot}}\left(r\right) - D_e\left\lbrace\left[1 - e^{-\alpha\left(r -r_c\right)}\right]^2 - 1\right\rbrace \right\vert^2 dr
$$

with respect to the parameters $D_e$, $\alpha$ and $r_c$.

In [ ]:
indices = (E < 0.)
result = scipy.optimize.curve_fit(morse, r[indices], E[indices],
                                  p0=(De1, alpha1, rc1))
De2, alpha2, rc2 = result[0]
display(Markdown(r"""
Morse (b):

- $r_c = {:.2f}\;\mathrm{{Å}}$
- $\alpha = {:.2f}\;\mathrm{{Å^{{-1}}}}$
- $D_e = {:.2f}\;\mathrm{{eV}}$
""".format(rc2, alpha2, De2)))

## Morse (c): A potential that reproduces the position and depth of the minimum, and the fundamental vibration frequency of the dimer

In [ ]:
De3 = -E_min
Ktilde = sp.misc.derivative(sp.interpolate.interp1d(r, E), r_min,
                            dx=0.1, n=2, order=3)
alpha3 = np.sqrt(Ktilde / (2. * De3))
rc3 = r_min
display(Markdown(r"""
Morse (c):

- $r_c = {:.2f}\;\mathrm{{Å}}$
- $\alpha = {:.2f}\;\mathrm{{Å^{{-1}}}}$
- $D_e = {:.2f}\;\mathrm{{eV}}$
""".format(rc3, alpha3, De3)))

## Comparison of all results

In [ ]:
# Plot all of the results alongside the real thing
Des = [De1, De2, De3]
alphas = [alpha1, alpha2, alpha3] 
rcs = [rc1, rc2, rc3]

plt.figure()
plt.scatter(r, E, color=colors[0], label="DFT")
plt.plot(newr, newE, color=colors[0])
for i, (De, alpha, rc) in enumerate(zip(Des, alphas, rcs)):
    plt.plot(newr, morse(newr, De, alpha, rc), color=colors[i + 1],
             label="Morse ({})".format(alphabet[i]))
plt.axhline(y=0., lw=1., color="#666666", ls=":")
plt.xlabel(r"$r\;\left(\mathrm{\AA}\right)$")
plt.ylabel(r"$E_{\mathrm{pot}}\;\left(\mathrm{eV}\right)$")
plt.legend(loc="best")
plt.xlim(1., 4.)
plt.ylim(-4., 4.)
plt.tight_layout()